In [1]:
import re
import os
import json
import pandas as pd

In [2]:
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())

In [3]:
persony = pd.read_json(os.path.join("data", "persony_raw.json"))

In [4]:
persony

,Jméno,Medailon,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
0,Ondřej Mašek (1000),,None,None,None,None,None,None
1,kino Světozor (100019),None,None,None,None,None,None,None
2,"AD WOOD, s.r.o. (100038)",None,None,None,None,None,None,None
3,Hypermarket Film s.r.o. (100039),None,Hypermarket Film,None,None,None,None,None
4,Hypermarket Film s.r.o. (100039),None,Hypermarket Film,None,None,None,None,None
...,...,...,...,...,...,...,...,...
35810,Josef Dvořák (9998),"herec, divadelní podnikatel",Josef Voloďa Dvořák (rodné jméno),None,25.04.1942,"Horní Cerekev, Československo",None,None
35811,Antonín Dimitrov (99996),"výtvarník, grafik, scénograf, designér",None,None,27.02.1928,"Mšecké Žehrovice, Československo",27.12.2014,"Bobcaygeon, Kanada"
35812,Čechomor (person),None,None,None,None,None,None,None
35813,Kevin Mc Kidd (person),,None,None,None,None,None,None


In [5]:
def odnebovat(x):
    if x:
        if "nebo" in x:
            x = x.split("nebo")[1].strip()
    return x

In [6]:
def rozdel_misto(x):
    if type(x) == str:
        try:
            mesto = x.split(",")[0]
            stat = x.split(",")[1]
        except:
            mesto = None
            stat = x
    else:
        mesto = None
        stat = None
    return [mesto, stat]

## Doplnění dat z Wikidat

In [7]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Medailon,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18963,Stanislav Litera (35280),,None,None,None,None,None,None


In [8]:
with open(os.path.join("data_fixes", "data_z_wikidat.json"), encoding="utf-8") as data_z_wikidat:
    data_z_wikidat = json.loads(data_z_wikidat.read())

In [9]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val[0]

In [10]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum úmrtí"] = val[1]

In [11]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Medailon,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18963,Stanislav Litera (35280),,None,None,11.06.1931,None,21.09.2003,None


## Opravy

In [12]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Medailon,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18566,Eva Pecková (34563),"pedagožka, zpěvačka, tanečnice",Eva Kubičková (rodné jméno),None,09.12.9441,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika"


In [13]:
for key, val in opravy["data_narozeni"].items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val

In [14]:
for key, val in opravy["jmena"].items():
    persony.loc[persony["Jméno"] == key, "Jméno"] = val

In [15]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Medailon,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
18566,Eva Pecková (34563),"pedagožka, zpěvačka, tanečnice",Eva Kubičková (rodné jméno),None,,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika"


In [16]:
persony[persony["Jméno"] == "Adam Klemens (1779)"]

,Jméno,Medailon,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
11585,Adam Klemens (1779),"dirigent, hudební skladatel",None,None,14.01.1967,None,None,None


## Konverze formátů

In [17]:
persony["Datum narození"].sample(200).unique()

array([None, '23.10.1999', '04.04.1943', '17.11.1976', '30.01.1996',
       '30.10.1955', '1933', '09.04.1957', '03.05.1925', '11.10.1881',
       '20.02.1952', '19.05.1957', '19.06.1905', '12.01.1933', '1939',
       '19.08.1946', '26.10.1909', '1956', '1974', '01.03.1969',
       '26.04.1933', '1978', '12.02.1875', '01.06.1950', '1942',
       '18.08.1932', '12.12.1943', '11.04.1944', '11.05.1898',
       '12.05.1901', '15.04.1957', '12.08.1944', '11.07.1969',
       '21.06.1964', '12.02.1950', '24.04.1907', '02.10.1895',
       '26.02.1903', '04.07.1977', '13.10.1963', '1900', '21.11.1874',
       '1953', '19.08.1935', '1976', '22.11.1945', '5. 5. 1978',
       '27.09.1892', '29.05.1975', '03.09.1921', '18.11.1847',
       '27.06.1928', '30.01.1953', '17.04.1946', '06.08.1933',
       '31.03.1958', '1980', '29.04.1946', '13.03.1930', '13.04.1961',
       '23.12.1914', '20.06.1892', '1835', '23.12.1928', '24.07.1948',
       '26.02.2000', '01.05.1907', '19.07.1923', '03.09.1952'],
  

In [18]:
persony["Město narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[0])
persony["Stát narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[1])
persony["Město úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Stát úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Rok narození"] = pd.to_numeric(persony["Datum narození"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")
persony["Rok úmrtí"] = pd.to_numeric(persony["Datum úmrtí"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")

In [19]:
def na_data(sloupec):
    global persony
    persony["Pracovni"] = persony[sloupec]
    persony["Pracovni"] = persony["Pracovni"].apply(lambda x: odnebovat(x))
    for index, row in persony.iterrows():
        try:
            date = pd.to_datetime(row["Pracovni"], format = "%d.%m.%Y", errors="ignore")
        except ValueError:
            date = pd.to_datetime(row["Pracovni"], format = "%Y", errors="ignore")
        except ValueError:
            date = None
        persony.at[index, sloupec] = date
    persony = persony.drop(["Pracovni"], axis = 1)

In [20]:
na_data("Datum narození")
na_data("Datum úmrtí")

In [21]:
persony

,Jméno,Medailon,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození,Rok úmrtí
0,Ondřej Mašek (1000),,None,None,None,None,None,None,None,None,None,None,NaN,NaN
1,kino Světozor (100019),None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
2,"AD WOOD, s.r.o. (100038)",None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
3,Hypermarket Film s.r.o. (100039),None,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
4,Hypermarket Film s.r.o. (100039),None,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35810,Josef Dvořák (9998),"herec, divadelní podnikatel",Josef Voloďa Dvořák (rodné jméno),None,1942-04-25 00:00:00,"Horní Cerekev, Československo",None,None,Horní Cerekev,Československo,None,None,1942.0,NaN
35811,Antonín Dimitrov (99996),"výtvarník, grafik, scénograf, designér",None,None,1928-02-27 00:00:00,"Mšecké Žehrovice, Československo",2014-12-27 00:00:00,"Bobcaygeon, Kanada",Mšecké Žehrovice,Československo,Kanada,Kanada,1928.0,2014.0
35812,Čechomor (person),None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
35813,Kevin Mc Kidd (person),,None,None,None,None,None,None,None,None,None,None,NaN,NaN


In [22]:
persony.to_json(os.path.join("data", "persony.json"), orient="records")